In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from glob import glob
import numpy as np

Using TensorFlow backend.


In [3]:
batch_size = 16

# this is the configuration for training set 
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# For testing only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator, will read pictures found in subfolers of 'data/train'
data_generator = train_datagen.flow_from_directory(
        './train/',  # this is the target directory
        target_size=(64, 64),  # all images will be resized to 150x150
        color_mode = 'grayscale', # rgb
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels


Found 25000 images belonging to 2 classes.


In [4]:
X = list()
y = list()
batch_index = 0

while batch_index <= data_generator.batch_index:
    data = data_generator.next()
    for batch_data_X,batch_data_y in zip(data[0],data[1]):
        X.append(batch_data_X)
        y.append(batch_data_y)
    batch_index = batch_index + 1
    if batch_index>1000:
        break
del data_generator
X = np.asarray(X)
y = to_categorical(y, num_classes=2)

In [5]:
X.shape

(16016, 64, 64, 1)

In [6]:
n_imgs = X.shape[0]
n_cols = X.shape[1]*X.shape[2]

In [8]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3, 3), activation= 'relu',  input_shape=X.shape[1:]))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())

model.add(Dense(3,activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(2,activation='softmax',input_shape= (n_cols,) ))

In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
my_callback = EarlyStopping(patience=3)

In [11]:
model.fit(X, y,
        batch_size=150, 
        epochs=50,
        validation_split=0.2,
        callbacks =  [my_callback])
model.save_weights('first_try.h5')  # always save your weights after training or during training

Train on 12812 samples, validate on 3204 samples
Epoch 1/50
12812/12812 [==============================] - 30s 2ms/step - loss: 0.6897 - acc: 0.5302 - val_loss: 0.6817 - val_acc: 0.5546
Epoch 2/50
12812/12812 [==============================] - 28s 2ms/step - loss: 0.6706 - acc: 0.5913 - val_loss: 0.6581 - val_acc: 0.5943
Epoch 3/50
12812/12812 [==============================] - 28s 2ms/step - loss: 0.6566 - acc: 0.6197 - val_loss: 0.6478 - val_acc: 0.6096
Epoch 4/50
12812/12812 [==============================] - 28s 2ms/step - loss: 0.6384 - acc: 0.6451 - val_loss: 0.6270 - val_acc: 0.6539
Epoch 5/50
12812/12812 [==============================] - 30s 2ms/step - loss: 0.6248 - acc: 0.6560 - val_loss: 0.6185 - val_acc: 0.6863
Epoch 6/50
12812/12812 [==============================] - 28s 2ms/step - loss: 0.6171 - acc: 0.6659 - val_loss: 0.6090 - val_acc: 0.6944
Epoch 7/50
12812/12812 [==============================] - 29s 2ms/step - loss: 0.6097 - acc: 0.6747 - val_loss: 0.6043 - val_acc:

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 38403     
_________________________________________________________________
dropout_1 (Dropout)          (None, 3)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 8         
Total params: 38,731
Trainable params: 38,731
Non-trainable params: 0
_________________________________________________________________
